<a href="https://colab.research.google.com/github/dinaldoap/jit-sdp-nn/blob/master/notebook/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random
from scipy.stats import mstats

In [54]:
df = pd.read_csv('https://raw.githubusercontent.com/dinaldoap/jit-sdp-data/master/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [57]:
#df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [58]:
print(x[:1])
print(y[:1])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[0]


In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [60]:
print(x_train[:1])
print(x_val[:1])
print(x_test[:1])

[[0.00000000e+00 7.00000000e+00 7.00000000e+00 7.00000000e+00
  2.64160417e+00 9.00000000e+00 9.00000000e+00 4.26428571e+02
  1.00000000e+02 9.25925926e-05 1.00000000e+00 5.17100000e+03
  3.02272705e+01 1.47271429e+03 0.00000000e+00 7.00000000e+00
  7.00000000e+00 7.00000000e+00 2.64160417e+00 9.00000000e+00
  9.00000000e+00 4.26428571e+02 1.00000000e+02 9.25925926e-05
  1.00000000e+00 5.17100000e+03 3.02272705e+01 1.47271429e+03
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[0.00000000e+00 8.00000000e+00 8.00000000e+00 8.00000000e+00
  2.73215889e+00 1.10000000e+01 1.10000000e+01 1.58625000e+02
  5.00000000e+00 1.96759259e-04 1.00000000e+00 4.17300000e+03
  2.45570735e+02 8.19125000e+02 0.00000000e+00 8.00000000e+00
  8.00000000e+00 8.00000000e+00 2.73215889e+00 1.10000000e+01
  1.10000000e+01 1.58625000e+02 5.00000000e+00 1.96759259e-04
  1.00000000e+00 4.17300000e+03 2.45570735e+02 8.19125000e+02
  0.00000000e+00 0.00000000e+00 0.00

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [62]:
print(mean_train)
print(std_train)

[2.90871477e-01 1.61432635e+00 2.60858547e+00 4.18908715e+00
 7.43390526e-01 5.00800621e+01 2.96331523e+01 8.11968929e+02
 2.32963538e+01 9.76994760e+01 2.02296354e+00 1.73095790e+03
 6.53773667e+01 8.28714959e+02 2.90871477e-01 1.61432635e+00
 2.60858547e+00 4.18908715e+00 7.43390526e-01 5.00800621e+01
 2.96331523e+01 8.11968929e+02 2.32963538e+01 9.76994760e+01
 2.02296354e+00 1.73095790e+03 6.53773667e+01 8.28714959e+02
 2.90871477e-01 1.47142488e-01 3.46521852e-02 4.71373157e-01
 1.51538660e-02 4.08068270e-02]
[4.54164354e-01 1.62736223e+00 6.93052365e+00 2.56616350e+01
 1.13236779e+00 5.86250473e+02 6.19852440e+02 1.15236538e+03
 2.84287684e+01 2.37802272e+02 7.67653474e+00 2.11471132e+03
 6.96820390e+01 1.34456679e+03 4.54164354e-01 1.62736223e+00
 6.93052365e+00 2.56616350e+01 1.13236779e+00 5.86250473e+02
 6.19852440e+02 1.15236538e+03 2.84287684e+01 2.37802272e+02
 7.67653474e+00 2.11471132e+03 6.96820390e+01 1.34456679e+03
 4.54164354e-01 3.54247902e-01 1.82897270e-01 4.99179

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [64]:
print(x_train[:1])
print(x_val[:1])
print(x_test[:1])

[[-0.64045422  3.30944983  0.63363387  0.10953756  1.67632253 -0.07007254
  -0.0332872  -0.33456434  2.69809951 -0.41084293 -0.1332585   1.62671948
  -0.50443553  0.47896418 -0.64045422  3.30944983  0.63363387  0.10953756
   1.67632253 -0.07007254 -0.0332872  -0.33456434  2.69809951 -0.41084293
  -0.1332585   1.62671948 -0.50443553  0.47896418 -0.64045422 -0.41536587
  -0.18946256  1.05899079 -0.1240445  -0.20625922]]
[[-0.64045422  3.92394118  0.77792311  0.14850624  1.75629188 -0.06666103
  -0.03006063 -0.566959   -0.64358587 -0.4108425  -0.1332585   1.15478745
   2.58593708 -0.00713238 -0.64045422  3.92394118  0.77792311  0.14850624
   1.75629188 -0.06666103 -0.03006063 -0.566959   -0.64358587 -0.4108425
  -0.1332585   1.15478745  2.58593708 -0.00713238 -0.64045422 -0.41536587
  -0.18946256  1.05899079 -0.1240445  -0.20625922]]
[[-0.64045422 -0.37749822 -0.23210158 -0.12427451 -0.65649211 -0.08371859
  -0.0461935  -0.5041534  -0.74911278 -0.41080896 -0.1332585   0.26435859
   2.4362

In [0]:
def calc_fading_weights(size, fading_factor):  
  fading_weights = reversed(range(size))
  return [fading_factor**x for x in fading_weights]
  
def create_sampler(y):
  n_samples = len(y)
  fading_count = calc_fading_weights(n_samples, 0.9999) 
  total = np.sum(fading_count)
  bug = np.sum(fading_count * y)
  normal = total - bug
  class_weights = total / [normal, bug]
  class_weights = class_weights[y]

  instance_weights = calc_fading_weights(n_samples, 0.9999) 
  weights = instance_weights * class_weights
  return data.WeightedRandomSampler(weights=weights, num_samples=n_samples, replacement=True)

In [0]:
sampler = create_sampler(y_train)

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, sampler=sampler)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [69]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:1])
print(targets[:1])

inputs, targets = next(iter(val_dataloader))
print(inputs[:1])
print(targets[:1])

inputs, targets = next(iter(test_dataloader))
print(inputs[:1])
print(targets[:1])

tensor([[-0.6405, -0.3775,  0.0565, -0.0463,  0.5542, -0.0718, -0.0446, -0.6375,
         -0.7491,  0.0449, -0.1333,  1.8159,  2.1982,  2.8487, -0.6405, -0.3775,
          0.0565, -0.0463,  0.5542, -0.0718, -0.0446, -0.6375, -0.7491,  0.0449,
         -0.1333,  1.8159,  2.1982,  2.8487, -0.6405, -0.4154, -0.1895, -0.9443,
         -0.1240,  4.8483]], dtype=torch.float64)
tensor([1], dtype=torch.int8)
tensor([[-0.6405,  3.9239,  0.7779,  0.1485,  1.7563, -0.0667, -0.0301, -0.5670,
         -0.6436, -0.4108, -0.1333,  1.1548,  2.5859, -0.0071, -0.6405,  3.9239,
          0.7779,  0.1485,  1.7563, -0.0667, -0.0301, -0.5670, -0.6436, -0.4108,
         -0.1333,  1.1548,  2.5859, -0.0071, -0.6405, -0.4154, -0.1895,  1.0590,
         -0.1240, -0.2063]], dtype=torch.float64)
tensor([0], dtype=torch.int8)
tensor([[-0.6405, -0.3775, -0.2321, -0.1243, -0.6565, -0.0837, -0.0462, -0.5042,
         -0.7491, -0.4108, -0.1333,  0.2644,  2.4362,  0.8481, -0.6405, -0.3775,
         -0.2321, -0.1243, -0.

In [0]:
class Classifier(nn.Module):
  def __init__(self, input_size, hidden_size, drop_prob, epoch=None, val_gmean=None):
    super(Classifier, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.drop_prob = drop_prob
    self.epoch = epoch
    self.val_gmean = val_gmean
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
    self.dropout = nn.Dropout(drop_prob)
  
  def forward(self, x):
    x = torch.relu(self.fc1(x))
    x = self.dropout(x)
    x = torch.sigmoid(self.fcout(x))
    return x

In [43]:
classifier = Classifier(input_size=x.shape[1], hidden_size=x.shape[1], drop_prob=0.5)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=34, bias=True)
  (fcout): Linear(in_features=34, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'drop_prob': classifier.drop_prob,
      'val_gmean': classifier.val_gmean,
      'epoch': classifier.epoch,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['drop_prob'], checkpoint['epoch'], checkpoint['val_gmean'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [46]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.4335],
        [0.4339],
        [0.6298]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.003)

In [0]:
def calc_loss(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    loss = 0
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      batch_loss = criterion(outputs.squeeze(), targets.float())
      loss += batch_loss.item()

    return loss / len(dataloader)

def calc_recalls(targets, predictions):
  n_classes = 2
  confusion_matrix, _, _ = np.histogram2d(targets.detach().cpu().numpy(), predictions.squeeze().detach().cpu().numpy(), bins=n_classes)
  recalls = np.diag(confusion_matrix) / ((np.sum(confusion_matrix, axis=1)) + 1e-12)
  return recalls

def calc_gmean(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    recalls = np.zeros((2))    
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      recalls += calc_recalls(targets, predictions)
    recalls = recalls / len(dataloader)
    return mstats.gmean(recalls), recalls

In [49]:
train_loss = 0
epochs = 1000
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss = train_loss / len(train_dataloader)
  val_loss = calc_loss(classifier, val_dataloader)
  val_gmean, _ = calc_gmean(classifier, val_dataloader)
  print('Epoch: {}, Train loss: {}, Val loss: {}, Val g-mean: {}'.format(epoch, train_loss, val_loss, val_gmean))

  if classifier.val_gmean is None or val_gmean > classifier.val_gmean:
    classifier.epoch = epoch
    classifier.val_gmean = val_gmean
    save_classifier(classifier)

classifier.epoch = epoch
classifier.val_gmean = val_gmean

Epoch: 0, Train loss: 0.665351572789644, Val loss: 0.6297540178424433, Val g-mean: 0.6771454876139964
Epoch: 1, Train loss: 0.6301407093321518, Val loss: 0.6088929023397597, Val g-mean: 0.6744397668512775
Epoch: 2, Train loss: 0.6070351879704292, Val loss: 0.5944170575392874, Val g-mean: 0.6777316801477715
Epoch: 3, Train loss: 0.6006738927304438, Val loss: 0.5732412165717075, Val g-mean: 0.6875771639974287
Epoch: 4, Train loss: 0.5930105771577664, Val loss: 0.5761198299495798, Val g-mean: 0.6837619321887003
Epoch: 5, Train loss: 0.591766050839519, Val loss: 0.5694950905285383, Val g-mean: 0.698338623490053
Epoch: 6, Train loss: 0.5848290306273284, Val loss: 0.548805841489842, Val g-mean: 0.6894905770514114
Epoch: 7, Train loss: 0.57570003831058, Val loss: 0.5553134287658491, Val g-mean: 0.7038496588569324
Epoch: 8, Train loss: 0.5774954022450586, Val loss: 0.5464294380263278, Val g-mean: 0.69983028439133
Epoch: 9, Train loss: 0.5708784980248028, Val loss: 0.5294881412073186, Val g-mea

In [0]:
def evaluate(classifier):
  if torch.cuda.is_available():
    classifier = classifier.cuda()
  train_gmean, _ = calc_gmean(classifier, train_dataloader)
  test_gmean, test_recalls = calc_gmean(classifier, test_dataloader)
  print('Epoch: {}'.format(classifier.epoch))
  print('Train g-mean: {}'.format(train_gmean.item()))
  print('Val g-mean: {}'.format(classifier.val_gmean))
  print('Test g-mean: {}, recalls: {}'.format(test_gmean.item(), test_recalls))

In [51]:
print('Best classifier')
evaluate(load_classifier())

Best classifier
Epoch: 963
Train g-mean: 0.783911177068493
Val g-mean: 0.7787225170888533
Test g-mean: 0.748405987385661, recalls: [0.76141803 0.73561631]


In [52]:
print('Last classifier')
evaluate(classifier)

Last classifier
Epoch: 999
Train g-mean: 0.7764929398153447
Val g-mean: 0.7559171093920765
Test g-mean: 0.7472858153777943, recalls: [0.74394294 0.75064371]
